# Welcome to reddit-spectra! - Vigaash & Salik

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('text_comments.csv')
df.head

/Users/vigaashsivasothy/Documents/School/CSCD25/cscd25/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3508: DtypeWarning: Columns (2,3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


<bound method NDFrame.head of                   id  score    link_id                author  \
0         t1_ftjl56l    4.0  t3_gzv6so             mega_trex   
1         t1_ftjpxmc    6.0  t3_gzv6so             [deleted]   
2         t1_gzzxfyt   22.0  t3_nodb9e             divadream   
3         t1_gzzy7nc   92.0  t3_no6qaj  Ziegenkoennenfliegen   
4         t1_h00tpbp   82.0  t3_nolx7p       meowrottenralph   
...              ...    ...        ...                   ...   
46413720  t1_ew3s0p8    1.0  t3_cmccmo                  AJTK   
46413721  t1_ennpzrt    1.0  t3_bp2f3w          imdelirious3   
46413722  t1_f02psre    5.0  t3_d3f1tn      NocapNightingale   
46413723  t1_f5t4bhw    1.0  t3_dp6jz9              jag-engr   
46413724  t1_errythg    1.0  t3_c3ad9x             Mercbeast   

                  subreddit  \
0         BeautyGuruChatter   
1         BeautyGuruChatter   
2         BeautyGuruChatter   
3         BeautyGuruChatter   
4         BeautyGuruChatter   
...            

#### Initial cleaning. We are only concerned with four main columns, the subreddit, score, the text body and the date. We removed the rest of the columns to ensure anonymity of the users and reduce unneeded information.

In [4]:
df2 = df[['subreddit', 'score', 'body', 'created_utc']]
df2

,subreddit,score,body,created_utc
0,BeautyGuruChatter,4.0,Does anyone have a good cruelty free one? The ...,1.591756e+09
1,BeautyGuruChatter,6.0,(stares at my soft glam i've had for like 3 ye...,1.591758e+09
2,BeautyGuruChatter,22.0,When Jen’s initial reactions came out to the s...,1.622398e+09
3,BeautyGuruChatter,92.0,I think you mean a \n>Highschool *fucking* bully,1.622399e+09
4,BeautyGuruChatter,82.0,Ugh. I was honestly hoping that this brand wou...,1.622415e+09
...,...,...,...,...
46413720,SquaredCircle,1.0,"Did people forget Finn had to face Corbin, Jin...",1.565093e+09
46413721,AskReddit,1.0,I hope,1.557950e+09
46413722,TheStrokes,5.0,I will literally cry if you're right dude xD L...,1.568336e+09
46413723,ChoosingBeggars,1.0,Wow. That's incredibly hostile for no reason.,1.572450e+09


#### We will now check, which of the most polarized subreddits we have data for.

In [5]:
df_leftwing = df2[df2['subreddit'].isin(['democrats', 'EnoughLibertarianSpam', 'hillaryclinton', 'progressive', 'BlueMidterm2018', 'EnoughHillHate', 'Enough_Sanders_Spam', 'badwomensanatomy'])]

In [6]:
print("Number of entries from r/democrats: ", df_leftwing[df_leftwing['subreddit'] == 'democrats'].shape[0])
print("Number of entries from r/EnoughLibertarianSpam: ", df_leftwing[df_leftwing['subreddit'] == 'EnoughLibertarianSpam'].shape[0])
print("Number of entries from r/hillaryclinton: ", df_leftwing[df_leftwing['subreddit'] == 'hillaryclinton'].shape[0])
print("Number of entries from r/progressive: ", df_leftwing[df_leftwing['subreddit'] == 'progressive'].shape[0])
print("Number of entries from r/BlueMidterm2018: ", df_leftwing[df_leftwing['subreddit'] == 'BlueMidterm2018'].shape[0])
print("Number of entries from r/EnoughHillHate: ", df_leftwing[df_leftwing['subreddit'] == 'EnoughHillHate'].shape[0])
print("Number of entries from r/Enough_Sanders_Spam: ", df_leftwing[df_leftwing['subreddit'] == 'Enough_Sanders_Spam'].shape[0])
print("Number of entries from r/badwomensanatomy: ", df_leftwing[df_leftwing['subreddit'] == 'badwomensanatomy'].shape[0])

Number of entries from r/democrats:  4446
Number of entries from r/EnoughLibertarianSpam:  1462
Number of entries from r/hillaryclinton:  0
Number of entries from r/progressive:  0
Number of entries from r/BlueMidterm2018:  0
Number of entries from r/EnoughHillHate:  0
Number of entries from r/Enough_Sanders_Spam:  16831
Number of entries from r/badwomensanatomy:  12163


In [7]:
df_rightwing = df2[df2['subreddit'].isin(['Conservative', 'The_Donald', 'TrueChristian', 'NoFapChristians', 'Mr_Trump', 'metacanada', 'conservatives', 'new_right'])]

In [8]:
print("Number of entries from r/Conservative: ", df_rightwing[df_rightwing['subreddit'] == 'Conservative'].shape[0])
print("Number of entries from r/The_Donald: ", df_rightwing[df_rightwing['subreddit'] == 'The_Donald'].shape[0])
print("Number of entries from r/TrueChristian: ", df_rightwing[df_rightwing['subreddit'] == 'TrueChristian'].shape[0])
print("Number of entries from r/NoFapChristians: ", df_rightwing[df_rightwing['subreddit'] == 'NoFapChristians'].shape[0])
print("Number of entries from r/Mr_Trump: ", df_rightwing[df_rightwing['subreddit'] == 'Mr_Trump'].shape[0])
print("Number of entries from r/metacanada: ", df_rightwing[df_rightwing['subreddit'] == 'metacanada'].shape[0])
print("Number of entries from r/conservatives: ", df_rightwing[df_rightwing['subreddit'] == 'conservatives'].shape[0])
print("Number of entries from r/new_right: ", df_rightwing[df_rightwing['subreddit'] == 'new_right'].shape[0])

Number of entries from r/Conservative:  76876
Number of entries from r/The_Donald:  0
Number of entries from r/TrueChristian:  6558
Number of entries from r/NoFapChristians:  0
Number of entries from r/Mr_Trump:  0
Number of entries from r/metacanada:  6195
Number of entries from r/conservatives:  3572
Number of entries from r/new_right:  0


#### From the analysis above we only have data from four subreddits from the left and right wing, we will choose these four from each to do our LDA analysis.

In [9]:
df_leftwing = df2[df2['subreddit'].isin(['democrats', 'EnoughLibertarianSpam', 'Enough_Sanders_Spam', 'badwomensanatomy'])]
df_rightwing = df2[df2['subreddit'].isin(['Conservative', 'TrueChristian', 'metacanada', 'conservatives'])]

#### We will now do some additional data cleaning by removing duplicate entries and removing any text bodies with '[removed]'/'[deleted]' as our data set includes removed and deleted comments.

In [13]:
df_rightwing = df_rightwing.drop_duplicates()
df_leftwing = df_leftwing.drop_duplicates()

df_rightwing = df_rightwing[df.body != '[removed]']
df_leftwing = df_leftwing[df.body != '[removed]']

df_rightwing = df_rightwing[df.body != '[deleted]']
df_leftwing = df_leftwing[df.body != '[deleted]']

/var/folders/95/n2pn5_2s6hs50t59pkmwqp9c0000gn/T/ipykernel_1573/3409734139.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_rightwing = df_rightwing[df.body != '[removed]']
/var/folders/95/n2pn5_2s6hs50t59pkmwqp9c0000gn/T/ipykernel_1573/3409734139.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_leftwing = df_leftwing[df.body != '[removed]']
/var/folders/95/n2pn5_2s6hs50t59pkmwqp9c0000gn/T/ipykernel_1573/3409734139.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_rightwing = df_rightwing[df.body != '[deleted]']
/var/folders/95/n2pn5_2s6hs50t59pkmwqp9c0000gn/T/ipykernel_1573/3409734139.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_leftwing = df_leftwing[df.body != '[deleted]']


In [14]:
df_rightwing

,subreddit,score,body,created_utc
4335884,conservatives,1.0,Federal lawmakers don't make a habit of going ...,1.606253e+09
4335885,conservatives,-1.0,"Let’s face it, every politician has something ...",1.606268e+09
4335886,conservatives,1.0,Are you that locked in a DNC bubble. Many sta...,1.609793e+09
4335887,conservatives,-20.0,I think it's probably warranted. Better safe t...,1.609802e+09
4335889,conservatives,1.0,"1. It's 30,000.\n\n2. This proves the point. H...",1.606283e+09
...,...,...,...,...
46407933,TrueChristian,10.0,"Heh. ""Bound to it"".",1.554425e+09
46408517,metacanada,3.0,"No man, you've been brainwashed by leftist med...",1.549636e+09
46409701,TrueChristian,3.0,"Yes thank for you for clarification, you can t...",1.575603e+09
46410807,Conservative,14.0,This political correctness racket is starting ...,1.574919e+09


In [15]:
df_leftwing

,subreddit,score,body,created_utc
4058133,EnoughLibertarianSpam,10.0,**[Swill milk scandal](https://en.wikipedia.or...,1.607839e+09
4058134,EnoughLibertarianSpam,1.0,"Idk about the tax code stuff, I mean plenty of...",1.607881e+09
4058135,EnoughLibertarianSpam,17.0,"Particularly vile, because of the philosophica...",1.607889e+09
4058136,EnoughLibertarianSpam,4.0,"No, tell me, who’s freedom do you value?",1.596058e+09
4058137,EnoughLibertarianSpam,217.0,"for some reason, the way he licks his fingers ...",1.607900e+09
...,...,...,...,...
46392932,Enough_Sanders_Spam,4.0,Does it smell as nice as it looks?,1.575048e+09
46395096,badwomensanatomy,1.0,No,1.558772e+09
46398231,Enough_Sanders_Spam,9.0,I don't spend much time there...how so?,1.569704e+09
46400313,EnoughLibertarianSpam,14.0,Anyone remember the name of that neighbor who ...,1.563605e+09
